# Importing all the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import read_csv
import seaborn as sns

# Let us load all the data into pandas dataframe from csv files

In [ ]:
item_categories=read_csv('../input/item_categories.csv')
items=read_csv('../input/items.csv')
sales_train=read_csv('../input/sales_train.csv')
sample_submission=read_csv('../input/sample_submission.csv')
shops=read_csv('../input/shops.csv')
test=read_csv('../input/test.csv')

# Let us look at the schema of all the dataframes to identify relationships between dataframes

In [ ]:
sales_train.info()

In [ ]:
shops.info()

In [ ]:
items.info()

In [ ]:
item_categories.info()

# Lets join sales_train, shops, items, item_categories to form a single data frame

In [ ]:
sales_train = pd.merge(sales_train,shops,on='shop_id',how='left')
sales_train = pd.merge(sales_train,items,on='item_id',how='left')
sales_train = pd.merge(sales_train,item_categories,on='item_category_id',how='left')
sales_train.info()

# Lets convert the  datatype of 'date' column from object to datetime

In [ ]:
sales_train['date']=pd.to_datetime(sales_train['date'])

# Creating New Feature 'Total Price'

In [ ]:
sales_train['total_price']=sales_train['item_price']*sales_train['item_cnt_day']

# Analysis Of One Categorical Variable

**Running Month**

In [ ]:
plt.figure(figsize=(35,10))
sns.countplot(x='date_block_num', data=sales_train);

There are more entries for Decembers of 2013 and 2014.

**Shop**

In [ ]:
plt.figure(figsize=(35,10))
sns.countplot(x='shop_id', data=sales_train)

Shop 31 has maximum number of entires.

**Item Category**

In [ ]:
plt.figure(figsize=(35,10))
sns.countplot(x='item_category_id', data=sales_train)

Item category 40 has highest number of entries

# Analysis of One Categorical Varriable and One Continous variable

** Month-wise aggregation on Item Count**

In [ ]:
sales_train_monthly = pd.DataFrame(sales_train.groupby(['date_block_num'])['item_cnt_day'].sum().reset_index().sort_values('item_cnt_day'))
plt.figure(figsize=(35,10))
sns.barplot(x="date_block_num", y="item_cnt_day", data=sales_train_monthly , order=sales_train_monthly['date_block_num'])

We see that the Number of items sold is peaking in Decembers due to the holidays. Number of Items sold in December 2013 is greater December 2014

** Month-wise aggregation on Total Price**

In [ ]:
sales_train_monthly_total_price = pd.DataFrame(sales_train.groupby(['date_block_num'])['total_price'].sum().reset_index().sort_values('total_price'))
plt.figure(figsize=(35,10))
sns.barplot(x="date_block_num", y="total_price", data=sales_train_monthly_total_price, order=sales_train_monthly_total_price['date_block_num'])

We see that the Revenue is also peaking in Decembers due to the holidays.
Revenue in December 2014 is greater December 2013. This is contrary to the number of items sold.

**Shop-wise aggregation on Item Count**

In [ ]:
sales_train_shop = pd.DataFrame(sales_train.groupby(['shop_id'])['item_cnt_day'].sum().reset_index().sort_values('item_cnt_day'))
plt.figure(figsize=(35,10))
sns.barplot(x="shop_id", y="item_cnt_day", data=sales_train_shop, order=sales_train_shop['shop_id'])

Shop 31 is selling highest number of items while Shop 36 and Shop 11 are selling very few items.

** Shop-wise aggregation on Total Price**

In [ ]:
sales_train_shop_total_price = pd.DataFrame(sales_train.groupby(['shop_id'])['total_price'].sum().reset_index().sort_values('total_price'))
plt.figure(figsize=(35,10))
sns.barplot(x="shop_id", y="total_price", data=sales_train_shop_total_price , order=sales_train_shop_total_price['shop_id'])

Shop 54 is 5th highest in terms of Revenue but 3rd highest in terms of Number of items sold.

**Item Category-wise aggregation on Item Count**

In [ ]:
sales_train_item_category_id = pd.DataFrame(sales_train.groupby(['item_category_id'])['item_cnt_day'].sum().reset_index().sort_values('item_cnt_day'))
plt.figure(figsize=(35,10))
sns.barplot(x="item_category_id", y="item_cnt_day", data=sales_train_item_category_id, order=sales_train_item_category_id['item_category_id'])

**Item Category-wise aggregation on Total Price**

In [ ]:
sales_train_item_category_id = pd.DataFrame(sales_train.groupby(['item_category_id'])['total_price'].sum().reset_index().sort_values('total_price'))
plt.figure(figsize=(35,10))
sns.barplot(x="item_category_id", y="total_price", data=sales_train_item_category_id, order=sales_train_item_category_id['item_category_id'])

Item Category 40 is being sold in max while Item Category 19 makes max Revenue

# Analysis of One Continous variable

**Item Price**

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot( y=sales_train['item_price'])

In [ ]:
def eldf(data):
    n=len(data)
    x = np.sort(data)
    y= np.arange(1, n+1)/n
    return x, y

In [ ]:
sns.set()
plt.figure(figsize=(35,10))
x,y = eldf(sales_train['item_price'])
plt.plot(x,y,marker='.',linestyle='none')
plt.xlabel('item_price')
plt.ylabel('ELDF')

The plot is distored due to the outliers. Lets try log scale in y-axis

In [ ]:
plt.figure(figsize=(10,10))
g = sns.boxplot( y=sales_train['item_price'])
g.set_yscale('log')
g

In [ ]:
sales_train['item_price'].describe()

The item price ranges from -1 to 307980. I am not sure what does item price -1 mean

**Item Count aggregated monthly**

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot( y=sales_train_monthly['item_cnt_day'])


In [ ]:
sns.set()
plt.figure(figsize=(35,10))
x,y = eldf(sales_train_monthly['item_cnt_day'])
plt.plot(x,y,marker='.',linestyle='none')
plt.xlabel('item_cnt_day')
plt.ylabel('ELDF')

In [ ]:
sales_train_monthly['item_cnt_day'].describe()

# Analysis of Two Categorical Variable and One Continous variable

**Categorical variables : Running Month, Shop 
Continous Variable : Item Count**

In [ ]:
plt.figure(figsize=(35,35))
sns.lmplot(data = sales_train, x = 'date_block_num', y = 'item_cnt_day',fit_reg=False, 
           hue = "shop_id",size = 20, aspect=1.5, scatter_kws = {'s':50}, )

Shop id 12 is notable here. On October 2015 it has sold 2169 item. Same shop sold 1000 items on Jan 2015. Must be a busy shop.

**Categorical variables : Running Month, Item Category 
Continous Variable : Item Count**

In [ ]:
plt.figure(figsize=(20,10))
sns.lmplot(data = sales_train, x = 'date_block_num', y = 'item_cnt_day',fit_reg=False, 
           hue = "item_category_id",size = 20, aspect=1.5, scatter_kws = {'s':50}, )

Item Category Id 9 has unusually high number of item counts on October 2015. The Item Category is 'Доставка товара' which means 'Delivery of Goods'. Not sure what is this.

# Analysis of Two Continous Variable and One Categorical variable

Used log on the columns since the numbers are quite hugh

In [ ]:
sales_train['item_price_log']=np.log(sales_train['item_price'])
sales_train['item_cnt_day_log']=np.log(sales_train['item_cnt_day'])

**Categorical variable : Shop ;
Continous Variable : Item Count, Item Price**

In [ ]:

sns.lmplot(data = sales_train, x = 'item_cnt_day_log', y = 'item_price_log',fit_reg=False, 
           hue = "shop_id",size = 20, aspect=1.5, scatter_kws = {'s':50}, )

**Categorical variable : Item Category ;
Continous Variable : Item Count, Item Price**

In [ ]:
sns.lmplot(data = sales_train, x = 'item_cnt_day_log', y = 'item_price_log',fit_reg=False, 
           hue = "item_category_id",size = 20, aspect=1.5, scatter_kws = {'s':50}, )

**Categorical variable : Running Month 
Continous Variable : Item Count, Item Price**

In [ ]:
sns.lmplot(data = sales_train, x = 'item_cnt_day_log', y = 'item_price_log',fit_reg=False, 
           hue = "date_block_num",size = 20, aspect=1.5, scatter_kws = {'s':50}, )

# Kernel Density Plots

**Running Month Vs Item Count**

In [ ]:
sns.jointplot(data = sales_train_monthly, x = 'date_block_num', y = 'item_cnt_day', kind='kde' , size=20)

Looks like a decline in the Number of Items sold over the months.

**Running Month Vs Total Price**

In [ ]:
plt.figure(figsize=(20,10))
sns.jointplot(data = sales_train_monthly_total_price, x = 'date_block_num', y = 'total_price', kind='kde' , size=20)

The data is not statistacally significant since p > 0.001

To be continued by removing the outliers........